In [7]:
import os
import time
import copy
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tqdm.auto import tqdm

# Set device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load dataset
data_dir = "/home/kevinluo/breast_density_classification/datasets"
transform = transforms.Compose([
    transforms.Resize((960, 960)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transform=transform)
    for x in ["train", "valid", "test"]
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=4)
    for x in ["train", "valid", "test"]
}

dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "valid", "test"]}
class_names = image_datasets["train"].classes

# Load the pre-trained ResNet101 model
model = models.resnet101(pretrained=False)
num_ftrs = model.fc.in_features

# Modify the model's output layer
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


### training with mlflow

In [ ]:
import mlflow
import mlflow.pytorch

def train_model(model, criterion, optimizer, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # 初始化 MLflow 實驗
    mlflow.set_experiment("Resnet101model_baseline")
    
    # 開始MLflow跟踪
    with mlflow.start_run():
        for epoch in range(num_epochs):
            print("Epoch {}/{}".format(epoch + 1, num_epochs))
            print("-" * 10)

            for phase in ["train", "valid"]:
                if phase == "train":
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0

                for inputs, labels in tqdm(dataloaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == "train"):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        if phase == "train":
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
                
                # 記錄 MLflow 指標
                mlflow.log_metric("{}_loss".format(phase), epoch_loss)
                mlflow.log_metric("{}_acc".format(phase), epoch_acc)

                if phase == "valid" and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print("Training complete in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60))
        print("Best validation accuracy: {:4f}".format(best_acc))

        model.load_state_dict(best_model_wts)
        
        # 儲存最佳模型
        mlflow.pytorch.log_model(model, "best_model")
        
    return model

trained_model = train_model(model, criterion, optimizer, num_epochs=200)


In [8]:
def train_model(model, criterion, optimizer, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch + 1, num_epochs))
        print("-" * 10)

        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))

            if phase == "valid" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print("Training complete in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60))
    print("Best validation accuracy: {:4f}".format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

trained_model = train_model(model, criterion, optimizer, num_epochs=200)


Epoch 1/200
----------


100%|██████████| 526/526 [08:05<00:00,  1.08it/s]


train Loss: 1.4048 Acc: 0.3928


100%|██████████| 54/54 [00:18<00:00,  2.99it/s]


valid Loss: 1.1398 Acc: 0.5163
Epoch 2/200
----------


...

100%|██████████| 526/526 [08:58<00:00,  1.02s/it]


train Loss: 0.0461 Acc: 0.9843


100%|██████████| 54/54 [00:18<00:00,  2.90it/s]


valid Loss: 3.9038 Acc: 0.5209
Epoch 96/200
----------


 95%|█████████▌| 502/526 [08:33<00:24,  1.02s/it]

### Test the model on the test set

In [ ]:
test_corrects = 0
predictions = []
ground_truth = []

with torch.no_grad():
    for inputs, labels in tqdm(dataloaders["test"]):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = trained_model(inputs)
        _, preds = torch.max(outputs, 1)

        test_corrects += torch.sum(preds == labels.data)
        predictions.extend(preds.cpu().numpy())
        ground_truth.extend(labels.cpu().numpy())
test_acc = test_corrects.double() / dataset_sizes["test"]
print("Test accuracy: {:.4f}".format(test_acc))

### metrics and plot

In [ ]:
cm = confusion_matrix(ground_truth, predictions)
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", square=True)
plt.xlabel("Predicted class")
plt.ylabel("True class")
plt.title("Confusion Matrix")
plt.show()

print(classification_report(ground_truth, predictions, target_names=class_names))